In [14]:
import numpy as np
import math
import sys
# import myenv
sys.path.append('../src/features/')
sys.path.append('../src/data/')
import data_preprocessing_from_yahoo_finance as dp
import myenv
from tqdm import tqdm
import matplotlib.pyplot as plt
import tensorflow as tf
from numba import cuda

In [15]:
result_np = dp.data_preprocessing_1(ticker_list_input=['PG','AAPL'], path='../data/raw/')
c = myenv.myenv(result_np)

working on PG
working on AAPL


In [16]:
class Agent:
    def __init__(self, sess, scope, mode, env=c, reuse=tf.AUTO_REUSE, lr=0.01):
        self.sess = sess
        self.gamma = np.float64(0.99)
        self.lr = lr
        self.scope = scope
        #############################################################
        # Inputs
        with tf.variable_scope(self.scope+'_inputs', reuse=reuse):
            self.price_tensor = tf.placeholder(dtype=tf.float64,
                              shape=[None]+env.observation_space_dimension['price_tensor'], name='price_tensor')
            self.weight = tf.placeholder(dtype=tf.float64,
                        shape=[None]+env.observation_space_dimension['weight'], name='weight')

        with tf.variable_scope(self.scope+'_network', reuse=reuse):
            # Conv Layers
            conv1 = tf.layers.conv2d(inputs=self.price_tensor,
                         filters=32,
                         kernel_size=[100,env.observation_space_dimension['price_tensor'][-1]], # Consider all stocks at once
                         strides=2,
                         padding='same', name='conv1')
            conv2 = tf.layers.conv2d(inputs=conv1,
                         filters=24,
                         kernel_size=[10,env.observation_space_dimension['price_tensor'][-1]],
                         strides=2,
                         padding='same', name='conv2')
            F = tf.reshape(conv2, [-1,np.prod(conv2.shape[1:])], name='flattern') # Flatten

            # Dense Layer
            dense1 = tf.layers.dense(inputs=F, units=100,
                        activation=tf.nn.relu, name='dense1')
            dense2 = tf.layers.dense(inputs=dense1, units=50,
                        activation=tf.nn.relu, name='dense2')
            # Concat weight tensor
            concat = tf.concat([dense2, self.weight], 1, name='concat')

            self.dense3 = tf.layers.dense(inputs=concat, units=100,
                        activation=tf.nn.relu, name='dense3')

            self.logits = tf.layers.dense(self.dense3, units=env.action_space_dimension, name='logit')


        with tf.variable_scope(self.scope+'_policy', reuse=reuse):
            self.new_weight = tf.nn.softmax(self.logits, name='new_weight') # Actor output
    
        with tf.variable_scope(self.scope+'_value', reuse=reuse):
            self.state_value = tf.layers.dense(inputs=self.dense3, units=1, # Critic Output
                        activation='linear', name='state_value')
            self.critic_train_op = tf.train.AdamOptimizer(self.lr)

            
        self.output = tf.concat([self.new_weight, self.state_value], 1)            

        
        self.actor_params = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope=self.scope+'_network')+ \
                            tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope=self.scope+'_output/new_weight')
        self.critic_params = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope=self.scope+'_network')+ \
                            tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope=self.scope+'_output/state_value')
        
        self.init = tf.global_variables_initializer()   
        self.sess.run(self.init)
        
    def step(self, price_tensor, weight):
        return self.sess.run(self.output, feed_dict={self.price_tensor:price_tensor, self.weight:weight})
        
    def get_value(self, S):
        return self.sess.run(self.state_value, feed_dict={self.price_tensor:S[0], self.weight:S[1]})
  
    def get_critic_variables(self):
        return tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope=self.scope+'_input') + \
                tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope=self.scope+'_network') + \
        tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope=self.scope+'_value/state_value')

In [17]:
class memory(): # https://github.com/MorvanZhou/Reinforcement-learning-with-tensorflow/blob/master/contents/9_Deep_Deterministic_Policy_Gradient_DDPG/DDPG.py
    def __init__(self, capacity, env):
        self.capacity = capacity
        self.env = env
        self.pointer = 0
        dim = 1+1+1+1# S,A,R,S_
        self.data = np.zeros(([capacity]+[dim])) # state, action, reward, next_state

    def store_transition(self, state_number, action_number, r, next_state_number):
        transition = [state_number, action_number, r, next_state_number]
        index = self.pointer % self.capacity
        self.data[index, :] = transition
        self.pointer += 1
        
    def sample(self, n):
        assert self.pointer >= self.capacity, 'Please fill the memory'
        indices = np.random.choice(self.capacity, size = n)
        sampled_data = self.data[indices, :]
        
        S=[]
        A=[]
        R=[]
        S_=[]
        for row in sampled_data:
            s = [self.env.all_prices_normalized[:,int(row[0]):int(row[0]+self.env.price_window),:], self.env.weights[int(row[0])] ]
            S.append(s) # current step -1
            A.append(self.env.weights[int(row[1]-1)])
            R.append(row[2])
            s_ = [self.env.all_prices_normalized[:,int(row[3]):int(row[3]+self.env.price_window),:], self.env.weights[int(row[3])] ]
            S_.append(s_)
        
        S = np.array(S)
        S_ = np.array(S_)
        
        result_S = [np.stack(S[:,0], axis=0), np.stack(S[:,1], axis=0)]
        result_A = A
        result_R = R
        result_S_ = [np.stack(S_[:,0], axis=0), np.stack(S_[:,1], axis=0)]
        return result_S, result_A, result_R, result_S_
#         return S,A,R,S_

In [18]:
tf.reset_default_graph()
price_tensor, weight,_,done = c.reset()
sess = tf.InteractiveSession()
a = Agent(scope='worker', mode='testing', sess=sess)
sess.run(a.init)
M = memory(capacity=50, env=c)
for i in tqdm(range(c.total_steps)):
#     if done: print(i)
#     print(c.current_step)
    S_number = c.current_step
#     sess.run(a.init)
#     agent_output = sess.run(a.build_policy(), 
#                       feed_dict={a.price_tensor:[price_tensor], 
#                                  a.weight:[weight]})
    agent_output = a.step([price_tensor], [weight])
    env_output = c.step(agent_output[0][:-1])
    price_tensor = env_output[0]
    weight = env_output[1]
    reward = env_output[2]
    done = env_output[3]
    M.store_transition(S_number, S_number, reward, S_number+1)

S,A,R,S_ = M.sample(10)

C:\Users\user\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\client\session.py:1645: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '

100%|██████████| 100/100 [00:01<00:00, 79.45it/s]


In [20]:
master = Agent(scope='master', mode='testing', sess=sess)
state_value_target_op = master.state_value
state_value_target_op = R +  tf.multiply(np.float64(.99), state_value_target_op)
state_value_target = state_value_target_op

state_value_op = a.state_value
state_value = state_value_op
temporal_difference = tf.subtract(state_value_target, state_value, name='TD_error')

# with tf.variable_scope('critic_loss', reuse=True):
critic_loss = tf.reduce_mean(tf.square(temporal_difference))
train_op = a.critic_train_op.minimize(critic_loss, var_list=a.get_critic_variables())
sess.run(train_op, feed_dict={a.price_tensor:S[0], a.weight:S[1], master.price_tensor:S_[0], master.weight:S_[1]})

In [21]:
S_t,A_t,R_t,S__t = M.sample(10)
for i in tqdm(range(100)):
    S,A,R,S_ = M.sample(20)
    sess.run(train_op, feed_dict={a.price_tensor:S[0], a.weight:S[1], master.price_tensor:S_[0], master.weight:S_[1]})
    if i % 10 == 0:
        print(sess.run(critic_loss, feed_dict={a.price_tensor:S_t[0], a.weight:S_t[1], master.price_tensor:S__t[0], master.weight:S__t[1]}))


  0%|          | 0/100 [00:00<?, ?it/s]

428.34829560486673



 10%|█         | 10/100 [00:06<00:59,  1.52it/s]

4269.336867107014



 20%|██        | 20/100 [00:13<00:52,  1.53it/s]

...


 80%|████████  | 80/100 [00:53<00:13,  1.53it/s]

6.37873152427276e-05



 90%|█████████ | 90/100 [00:59<00:06,  1.54it/s]

6.655081439651455e-05



100%|██████████| 100/100 [01:06<00:00,  1.53it/s]


In [24]:
np.cumsum(R)

array([ 0.00549842,  0.01099684,  0.01034081,  0.01616295,  0.01657702,
        0.01995032,  0.01744778,  0.02326992,  0.01972426,  0.015901  ,
        0.00373322,  0.00476779,  0.00122213,  0.00066501, -0.0007244 ,
       -0.00085074, -0.0002228 ,  0.00051793,  0.00560386,  0.0017806 ])

In [ ]:
tf.gradients(ys=loss,xs=a.critic_params)

In [ ]:
loss = tf.reduce_mean((sum(a.get_value(S_)) + R - sum(a.get_value(S)))**2)
tf.train.AdadeltaOptimizer(0.01).minimize(loss)

In [ ]:
["<tf.Variable 'network/conv1/kernel:0' shape=(100, 3, 3, 32) dtype=float64_ref>", 
 "<tf.Variable 'network/conv1/bias:0' shape=(32,) dtype=float64_ref>",
 "<tf.Variable 'network/conv2/kernel:0' shape=(10, 3, 32, 24) dtype=float64_ref>",
 "<tf.Variable 'network/conv2/bias:0' shape=(24,) dtype=float64_ref>",
 "<tf.Variable 'network/dense/kernel:0' shape=(14520, 100) dtype=float64_ref>", 
 "<tf.Variable 'network/dense/bias:0' shape=(100,) dtype=float64_ref>",
 "<tf.Variable 'network/dense_1/kernel:0' shape=(100, 50) dtype=float64_ref>",
 "<tf.Variable 'network/dense_1/bias:0' shape=(50,) dtype=float64_ref>",
 "<tf.Variable 'network/dense_2/kernel:0' shape=(53, 100) dtype=float64_ref>",
 "<tf.Variable 'network/dense_2/bias:0' shape=(100,) dtype=float64_ref>",
 "<tf.Variable 'network/dense_3/kernel:0' shape=(100, 3) dtype=float64_ref>",
 "<tf.Variable 'network/dense_3/bias:0' shape=(3,) dtype=float64_ref>",
 "<tf.Variable 'value/state_value/kernel:0' shape=(100, 1) dtype=float64_ref>",
 "<tf.Variable 'value/state_value/bias:0' shape=(1,) dtype=float64_ref>",
 "<tf.Variable 'Variable:0' shape=(10, 1) dtype=float64_ref>", 
 "<tf.Variable 'Variable_1:0' shape=(10, 1) dtype=float64_ref>"] 
loss Tensor("Mean_3:0", shape=(), dtype=float64).


In [ ]:
with tf.Session() as sess:
    tf.initialize_all_variables().run()
    t = a.train_critic(sess, S, A, R, S_)
    sess.run(t, feed_dict={a.price_tensor:S[0], a.weight:S[1]})

In [ ]:
price_tensor = tf.placeholder(dtype=tf.float64,
                  shape=[None]+c.observation_space_dimension['price_tensor'], name='price_tensor')
weight = tf.placeholder(dtype=tf.float64,
            shape=[None]+c.observation_space_dimension['weight'], name='weight')

next_state_value_ph = tf.placeholder(tf.float64, shape=[None,1])
r_ph = tf.placeholder(tf.float64, shape=[None, 1])

td_error = tf.reduce_mean(r_ph + 0.99*next_state_value_ph - a.state_value)


In [ ]:
tf.reset_default_graph()
state_value = tf.placeholder(tf.float64, [1,1])
R_ph = tf.placeholder(tf.float64, [None, ])  
next_state_value_ph = tf.placeholder(tf.float64, [None, ])      
next_state_value = sess.run(a.state_value, feed_dict={a.price_tensor:S_[0], a.weight:S_[1]}) 
        
td_error = tf.reduce_mean(R_ph + .99*next_state_value_ph - state_value)
critic_loss = tf.square(td_error)
        
critic_train_op = tf.train.AdamOptimizer(0.01).minimize(critic_loss)
        
sess.run(critic_train_op, feed_dict={a.price_tensor:S[0],
                                             a.weight:S[1], 
                                             R_ph:R,
                                             next_state_value_ph:next_state_value})   

In [ ]:
S,A,R,S_ = M.sample(10)
a.train_critic(sess,S,A,R,S_)

In [ ]:
price_tensor, weight,_,done = c.reset()
sess = tf.Session()
a = Agent(name='testing', mode='testing')
M = memory(capacity=50, env=c)

S_list = []
R_list = []
next_S_list= []
for i in tqdm(range(c.total_steps)):
    S_list.append(price_tensor)
#     if done: print(i)
#     print(c.current_step)
    sess.run(a.init)
    agent_output = sess.run(a.output, 
                      feed_dict={a.price_tensor:[price_tensor], 
                                 a.weight:[weight]})
    env_output = c.step(agent_output[0][:-1])
    price_tensor = env_output[0]
    weight = env_output[1]
    reward = env_output[2]
    done = env_output[3]
    R_list.append(reward)
    next_S_list.append(price_tensor)

In [ ]:
# obs --> Critic --> state_value

In [ ]:
sess.run(tf.multiply(R,a.gamma))